In [ ]:


# a) Import required libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

import os
# ==========================================================
# Step 1: Load the Pre-trained CNN Model (VGG16 without top)
# ==========================================================
base_model = VGG16(weights='vgg16_weights.h5',
                   include_top=False,
                   input_shape=(224, 224, 3))

print("✅ Pre-trained VGG16 model loaded successfully.")
# ==========================================================
# Step 2: Freeze lower convolutional layers
# ==========================================================
for layer in base_model.layers:
    layer.trainable = False

print("✅ Base layers frozen.")

# ==========================================================
# Step 3: Add custom classifier (fully connected layers)
# ==========================================================
# model = Sequential([
#     base_model,
#     Flatten(),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.3),
#     Dense(102, activation='softmax')
# ])
model = Sequential([
    base_model,
    Flatten(),
    Dense(64, activation='relu'),
    Dense(as many prersent in dataset or use 4, activation='softmax')
])

# ==========================================================
# Step 4: Compile the model
# ==========================================================
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# ==========================================================
# Step 5: Prepare the dataset (images in subfolders)
# ==========================================================
data_dir = "/content/drive/MyDrive/object"  # your dataset folder

train_datagen = ImageDataGenerator(
    #rescale: scales pixel values from [0,255] → [0,1] for stable training.
    rescale=1./255,
    validation_split=0.2  # 80% training, 20% validation
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

print("✅ Dataset loaded successfully.")


print("Number of classes:", train_generator.num_classes)

# ==========================================================
# Step 6: Train the classifier layers
# ==========================================================
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)
# ==========================================================
# Step 7: Fine-tune (unfreeze top layers)
# ==========================================================
# Unfreeze last 4 convolutional layers
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Re-compile with smaller learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("🔄 Fine-tuning top layers...")

fine_tune_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)
# ==========================================================
# Step 8: Evaluate and Save the model
# ==========================================================
loss, acc = model.evaluate(val_generator)
print(f"\n✅ Final Validation Accuracy: {acc*100:.2f}%")

model.save("vgg16_transfer_learning_final.h5")
print("✅ Model saved successfully as vgg16_transfer_learning_final.h5")

import matplotlib.pyplot as plt
x_test, y_test = next(val_generator)
import numpy as np
# Make predictions
predicted_value = model.predict(x_test)

# Get label names
labels = list(train_generator.class_indices.keys())

# Show example
n = 22  # choose any image index
plt.imshow(x_test[n])
plt.axis('off')
plt.title(f"Predicted: {labels[np.argmax(predicted_value[n])]}\nActual: {labels[np.argmax(y_test[n])]}")
plt.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Transfer Learning for object detection/classification leverages the pre-trained convolutional layers of a deep CNN (such as VGG16) that has already learned powerful feature maps from a large dataset like ImageNet. Instead of training a CNN from scratch, which is expensive and requires massive data, we load the pre-trained model without its top classifier and freeze its convolutional layers so that previously learned low-level and mid-level features (edges, textures, shapes) are preserved. On top of this frozen base network, we attach a custom classifier consisting of dense layers trained specifically on the new dataset. After initial training, we perform fine-tuning by unfreezing the top few convolution layers and training them with a very low learning rate to adapt higher-level feature representations to the new domain. This two-stage procedure—feature extraction followed by fine-tuning—enables high accuracy with limited data and significantly reduces training time. The final model can classify or detect objects effectively using the combined knowledge of a pre-trained backbone and task-specific classifier.